In [1]:
import pandas as pd
import pickle
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [2]:
train_df = pd.read_csv('Data/df_train.csv')

In [3]:
train_df.head()

,WC,DUW,GC,AF,C,LB,LKpa,TMin,DR,DB,Sett
0,6.0,17.0,12.0,38.5,0.0,1.0,200.0,42.0,44.835703,0.936833,0.082
1,6.0,17.0,12.0,38.5,0.0,1.0,200.0,80.0,44.835703,0.936833,0.091
2,6.0,17.0,12.0,38.5,0.0,1.0,200.0,108.0,44.835703,0.936833,0.100
3,6.0,17.0,12.0,38.5,0.0,1.0,200.0,91.0,44.835703,0.936833,0.115
4,6.0,17.0,12.0,38.5,0.0,1.0,200.0,165.0,44.835703,0.936833,0.118


In [4]:
# load the model from disk
def load_predict_model(filename, X,y):
    model = pickle.load(open(filename, 'rb'))
    #result = loaded_model.score(X_test, Y_test)
    scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_pred=model.predict(X)
    Columns_df=['WC', 'DUW' , 'GC' , 'AF', 'C' , 'LB' , 'LKpa' ,'TMin','DB']
    X = pd.DataFrame(X,columns=Columns_df)
    X['Y']=y    
    X['Y_hat']=y_pred
    return X

In [5]:
# load the model from disk
def load_predict_modelDNN(X,y):
    new_model = tf.keras.models.load_model('Model/DNN_Model.model')
    #result = loaded_model.score(X_test, Y_test)
    y_pred=new_model.predict(X)
    Columns_df=['WC', 'DUW' , 'GC' , 'AF', 'C' , 'LB' , 'LKpa' ,'TMin','DB']
    X=X.transpose(2,0,1).reshape(X.shape[0],-1)
    X = pd.DataFrame(X,columns=Columns_df)
    X['Y']=y    
    X['Y_hat']=y_pred
    return X

In [6]:
def prepare_data(data):
    scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
    train_df_Scal = scaler.fit_transform(data)
    X = pd.DataFrame(train_df_Scal)
    X_New = X.to_numpy()
    X_New= X_New.reshape(X_New.shape[0], X_New.shape[1], 1)
    return X_New

In [7]:
X=train_df.drop(['Sett','DR'], axis=1)
y=train_df.Sett

In [11]:
List_Model=['Ensembel_model.sav','Lasso_model.sav','LR_model.sav','MLPRegressorModel_model.sav'
            ,'Ridge_model.sav','SVR_model.sav']
for model in List_Model:
    data=load_predict_model('Model/'+model,X,y)
    path=r'DataPredict/'+model+'.csv'
    if not os.path.isfile(path):
        data.to_csv(path, index = False)
    

In [ ]:
X=train_df.drop(['Sett','DR'], axis=1)
y=train_df.Sett
X=prepare_data(X)
data=load_predict_modelDNN(X,y)
path=r'DataPredict/DNN_Model.csv'
if not os.path.isfile(path):
    data.to_csv(path, index = False)